In [ ]:
# !pip install lenskit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.2/227.2 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.2/68.2 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 91.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 65.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 79.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12

In [25]:
import pandas as pd

from lenskit.als import BiasedMFScorer
from lenskit.knn import ItemKNNScorer
from lenskit.batch import recommend
from lenskit.data import from_interactions_df, ItemListCollection, UserIDKey
from lenskit.metrics import NDCG, RBP, RecipRank, RunAnalysis, Recall, Precision
from lenskit.pipeline import topn_pipeline
from lenskit.splitting import SampleFrac, crossfold_users

In [3]:
df = pd.read_csv('../ALS/data/ratings.csv')
dataset = from_interactions_df(df, user_col='user_id', item_col='book_id', rating_col='rating')

In [7]:
model_als = BiasedMFScorer(features=50)

In [8]:
pipe = topn_pipeline(model_als)

In [9]:
# test data is organized by user
all_test = ItemListCollection(UserIDKey)
# recommendations will be organized by model and user ID
all_recs = ItemListCollection(["model", "user_id"])

for split in crossfold_users(dataset, 5, SampleFrac(0.2)):
    # collect the test data
    all_test.add_from(split.test)

    # train the pipeline, cloning first so a fresh pipeline for each split
    fit_als = pipe.clone()
    fit_als.train(split.train)
    # generate recs
    als_recs = recommend(fit_als, split.test.keys(), 100)
    all_recs.add_from(als_recs, model="ALS")

/usr/local/lib/python3.11/dist-packages/lenskit/als/_explicit.py:58: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at /pytorch/aten/src/ATen/SparseCsrTensorImpl.cpp:53.)
  rmat = rmat.to_sparse_csr()


In [11]:
ran = RunAnalysis()
ran.add_metric(NDCG())
ran.add_metric(RBP())
ran.add_metric(RecipRank())
ran.add_metric(Precision())
ran.add_metric(Recall())
results = ran.measure(all_recs, all_test)

In [13]:
results.list_metrics().groupby("model").mean()

,NDCG,RBP,RecipRank,Precision,Recall
model,,,,,
ALS,0.009026,0.002913,0.013173,0.003571,0.016426


# Test different Hyperparameter

## 1st Run

In [3]:
model_als2 = BiasedMFScorer(features=100, iterations=30, reg=0.01)

In [4]:
pipe = topn_pipeline(model_als2)

In [ ]:
# test data is organized by user
all_test = ItemListCollection(UserIDKey)
# recommendations will be organized by model and user ID
all_recs = ItemListCollection(["model", "user_id"])

for split in crossfold_users(dataset, 5, SampleFrac(0.2)):
    # collect the test data
    all_test.add_from(split.test)

    # train the pipeline, cloning first so a fresh pipeline for each split
    fit_als = pipe.clone()
    fit_als.train(split.train)
    # generate recs
    als_recs = recommend(fit_als, split.test.keys(), 100)
    all_recs.add_from(als_recs, model="ALS")

In [9]:
ran = RunAnalysis()
ran.add_metric(NDCG())
# ran.add_metric(RBP())
# ran.add_metric(RecipRank())
ran.add_metric(Precision())
ran.add_metric(Recall())
results = ran.measure(all_recs, all_test)

In [10]:
results.list_metrics().groupby("model").mean()

,NDCG,Precision,Recall
model,,,
ALS,0.002041,0.00319,0.001511


## 2nd run

In [9]:
model_als3 = BiasedMFScorer(embedding_size=10, iterations=25, reg=0.1, damping=10)

In [10]:
pipe = topn_pipeline(model_als3)

In [11]:
# test data is organized by user
all_test = ItemListCollection(UserIDKey)
# recommendations will be organized by model and user ID
all_recs = ItemListCollection(["model", "user_id"])

for split in crossfold_users(dataset, 5, SampleFrac(0.2)):
    # collect the test data
    all_test.add_from(split.test)

    # train the pipeline, cloning first so a fresh pipeline for each split
    fit_als = pipe.clone()
    fit_als.train(split.train)
    # generate recs
    als_recs = recommend(fit_als, split.test.keys(), 100)
    all_recs.add_from(als_recs, model="ALS")

In [12]:
ran = RunAnalysis()
ran.add_metric(NDCG())
# ran.add_metric(RBP())
# ran.add_metric(RecipRank())
ran.add_metric(Precision())
ran.add_metric(Recall())
results = ran.measure(all_recs, all_test)

In [13]:
results.list_metrics().groupby("model").mean()

,NDCG,Precision,Recall
model,,,
ALS,0.007946,0.003183,0.014585


## 3rd run

In [14]:
model_als4 = BiasedMFScorer(embedding_size=10, iterations=25, reg=1, damping=10)
pipe = topn_pipeline(model_als4)

In [15]:
# test data is organized by user
all_test = ItemListCollection(UserIDKey)
# recommendations will be organized by model and user ID
all_recs = ItemListCollection(["model", "user_id"])

for split in crossfold_users(dataset, 5, SampleFrac(0.2)):
    # collect the test data
    all_test.add_from(split.test)

    # train the pipeline, cloning first so a fresh pipeline for each split
    fit_als = pipe.clone()
    fit_als.train(split.train)
    # generate recs
    als_recs = recommend(fit_als, split.test.keys(), 100)
    all_recs.add_from(als_recs, model="ALS")

In [16]:
ran = RunAnalysis()
ran.add_metric(NDCG())
# ran.add_metric(RBP())
# ran.add_metric(RecipRank())
ran.add_metric(Precision())
ran.add_metric(Recall())
results = ran.measure(all_recs, all_test)

results.list_metrics().groupby("model").mean()

,NDCG,Precision,Recall
model,,,
ALS,0.007851,0.00314,0.014396


## 4th run

In [ ]:
model_als5 = BiasedMFScorer(embedding_size=50, iterations=20, reg=5, damping=5)
pipe = topn_pipeline(model_als5)

In [18]:
# test data is organized by user
all_test = ItemListCollection(UserIDKey)
# recommendations will be organized by model and user ID
all_recs = ItemListCollection(["model", "user_id"])

for split in crossfold_users(dataset, 5, SampleFrac(0.2)):
    # collect the test data
    all_test.add_from(split.test)

    # train the pipeline, cloning first so a fresh pipeline for each split
    fit_als = pipe.clone()
    fit_als.train(split.train)
    # generate recs
    als_recs = recommend(fit_als, split.test.keys(), 100)
    all_recs.add_from(als_recs, model="ALS")

In [19]:
ran = RunAnalysis()
ran.add_metric(NDCG())
# ran.add_metric(RBP())
# ran.add_metric(RecipRank())
ran.add_metric(Precision())
ran.add_metric(Recall())
results = ran.measure(all_recs, all_test)

results.list_metrics().groupby("model").mean()

,NDCG,Precision,Recall
model,,,
ALS,0.009021,0.003575,0.016412


## 5th run

In [20]:
model_als5 = BiasedMFScorer(embedding_size=50, iterations=20, reg=5, damping=5)
pipe = topn_pipeline(model_als5)

In [21]:
# test data is organized by user
all_test = ItemListCollection(UserIDKey)
# recommendations will be organized by model and user ID
all_recs = ItemListCollection(["model", "user_id"])

for split in crossfold_users(dataset, 5, SampleFrac(0.2)):
    # collect the test data
    all_test.add_from(split.test)

    # train the pipeline, cloning first so a fresh pipeline for each split
    fit_als = pipe.clone()
    fit_als.train(split.train)
    # generate recs
    als_recs = recommend(fit_als, split.test.keys(), 100)
    all_recs.add_from(als_recs, model="ALS")

In [22]:
ran = RunAnalysis()
ran.add_metric(NDCG())
# ran.add_metric(RBP())
# ran.add_metric(RecipRank())
ran.add_metric(Precision())
ran.add_metric(Recall())
results = ran.measure(all_recs, all_test)

results.list_metrics().groupby("model").mean()

,NDCG,Precision,Recall
model,,,
ALS,0.008971,0.003585,0.016434


# Test KNN

In [27]:
model_ii = ItemKNNScorer(k=20)
pipe_ii = topn_pipeline(model_ii)

In [28]:
# test data is organized by user
all_test = ItemListCollection(UserIDKey)
# recommendations will be organized by model and user ID
all_recs = ItemListCollection(["model", "user_id"])

for split in crossfold_users(dataset, 5, SampleFrac(0.2)):
    # collect the test data
    all_test.add_from(split.test)

    # do the same for item-item
    fit_ii = pipe_ii.clone()
    fit_ii.train(split.train)
    ii_recs = recommend(fit_ii, split.test.keys(), 100)
    all_recs.add_from(ii_recs, model="II")

In [29]:
ran = RunAnalysis()
ran.add_metric(NDCG())
# ran.add_metric(RBP())
# ran.add_metric(RecipRank())
ran.add_metric(Precision())
ran.add_metric(Recall())
results = ran.measure(all_recs, all_test)

results.list_metrics().groupby("model").mean()

,NDCG,Precision,Recall
model,,,
II,0.015597,0.005744,0.025355


## 2nd Run

In [30]:
model_ii = ItemKNNScorer(k=30)
pipe_ii = topn_pipeline(model_ii)

In [31]:
# test data is organized by user
all_test = ItemListCollection(UserIDKey)
# recommendations will be organized by model and user ID
all_recs = ItemListCollection(["model", "user_id"])

for split in crossfold_users(dataset, 5, SampleFrac(0.2)):
    # collect the test data
    all_test.add_from(split.test)

    # do the same for item-item
    fit_ii = pipe_ii.clone()
    fit_ii.train(split.train)
    ii_recs = recommend(fit_ii, split.test.keys(), 100)
    all_recs.add_from(ii_recs, model="II")

In [32]:
ran = RunAnalysis()
ran.add_metric(NDCG())
# ran.add_metric(RBP())
# ran.add_metric(RecipRank())
ran.add_metric(Precision())
ran.add_metric(Recall())
results = ran.measure(all_recs, all_test)

results.list_metrics().groupby("model").mean()

,NDCG,Precision,Recall
model,,,
II,0.014825,0.005445,0.023868
